# Program Aided Prompting Example

This notebook demonstrates a realistic workflow for **Program Aided Prompting (PAP)**. We will let a language model generate Python code to analyze a small sales dataset, execute that code programmatically, and then feed the results back to the model for a final summary.

## Setup

We use the `openai` Python package to access a GPT model. Set the `OPENAI_API_KEY` environment variable before running these cells.

In [ ]:
import os
import openai
import pandas as pd

openai.api_key = os.getenv('OPENAI_API_KEY')
assert openai.api_key, 'Please set OPENAI_API_KEY'


### Load the dataset

In [ ]:
df = pd.read_csv('sales_data.csv')
df.head()

## Step 1: Ask the model for analysis code
We prompt the model with a short description of the data and ask it to write Python code that returns:
1. Total sales across all categories
2. The name of the category with the highest sales
3. A bar chart visualizing the sales by category

In [ ]:
system_prompt = 'You are a helpful data analyst who writes clean Python code.'
user_prompt = f'The DataFrame `df` contains two columns: `category` and `sales`.
Please provide Python code that computes the total sales, finds the top category, and draws a bar chart of sales by category.'
response = openai.ChatCompletion.create(model='gpt-3.5-turbo', messages=[{'role':'system','content':system_prompt},{'role':'user','content':user_prompt}])
generated_code = response['choices'][0]['message']['content']
print(generated_code)

## Step 2: Execute the generated code
We use `exec` to run the code from the model in a restricted namespace and capture the resulting variables.

In [ ]:
namespace = {'df': df}
exec(generated_code, namespace)
total_sales = namespace.get('total_sales')
top_category = namespace.get('top_category')


## Step 3: Summarize the results
Finally, we feed the numerical results back into the model and request a short managerial summary.

In [ ]:
summary_prompt = f'The total sales were {total_sales} units. The top category was {top_category}. Write a short summary for a manager.'
summary_response = openai.ChatCompletion.create(model='gpt-3.5-turbo', messages=[{'role':'user','content':summary_prompt}])
summary = summary_response['choices'][0]['message']['content']
print(summary)

### Example prompts for faculty

Below are two prompts tailored for faculty at tier-2 engineering colleges in Telangana and Andhra Pradesh. You can copy these directly into ChatGPT or the integrated code interpreter.
1. **Placement analysis**: `The DataFrame df contains columns department and placement_percent. Write Python code that computes the overall average placement rate and draws a bar chart by department.`
2. **Exam results**: `The DataFrame df has columns student, subject and marks. Provide Python code that calculates the average marks for each subject and lists the top scoring student.`